In [2]:
'''
Challenge link
https://preppindata.blogspot.com/2023/03/2023-week-12-regulatory-reporting.html
'''

import pandas as pd
import numpy as np
import datetime as datetime

In [3]:
uk_holiday = pd.read_excel("New Customer Reporting.xlsx", sheet_name=0)
new_cust = pd.read_excel("New Customer Reporting.xlsx", sheet_name=1)
roi = pd.read_excel("New Customer Reporting.xlsx", sheet_name=2)

uk_holiday.head(5)

,Year,Date,Bank holiday
0,2022.0,NaT,NaN
1,NaN,2023-12-27,Christmas Day (substitute day)
2,NaN,2023-12-26,Boxing Day
3,NaN,2023-09-19,Bank Holiday for the State Funeral of Queen El...
4,NaN,2023-08-29,Summer bank holiday


In [4]:
new_cust.head(5)

,Date,New Customers
0,2021-12-31,4
1,2022-01-01,4
2,2022-01-02,5
3,2022-01-03,8
4,2022-01-04,3


In [5]:
roi.head(5)

,Reporting Month,Reporting Day,New Customers,Reporting Date
0,2023-09-01,1,6,2023-08-31
1,2023-09-01,2,1,2023-09-01
2,2023-09-01,3,11,2023-09-04
3,2023-09-01,4,3,2023-09-05
4,2023-09-01,5,4,2023-09-06


### Fill down the years and create a date field for the UK bank holidays

In [6]:
uk_holiday.info()
uk_holiday.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Year          2 non-null      float64       
 1   Date          19 non-null     datetime64[ns]
 2   Bank holiday  19 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 632.0+ bytes


,Year,Date,Bank holiday
0,2022.0,NaT,NaN
1,NaN,2023-12-27,Christmas Day (substitute day)
2,NaN,2023-12-26,Boxing Day
3,NaN,2023-09-19,Bank Holiday for the State Funeral of Queen El...
4,NaN,2023-08-29,Summer bank holiday


In [7]:
y = 0
for i in range(len(uk_holiday)):
    if np.isnan(uk_holiday.loc[i,'Year']) == True:
        m = uk_holiday.loc[1,'Date'].month
        d = uk_holiday.loc[1,'Date'].day
        uk_holiday.loc[i,'Date'] = datetime.datetime(int(y),m,d)
    else:
        y = uk_holiday.loc[i,'Year']
        uk_holiday.drop(index=i, inplace=True)

uk_holiday.drop(columns='Year', inplace=True)
uk_holiday.info()
uk_holiday.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 1 to 20
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          19 non-null     datetime64[ns]
 1   Bank holiday  19 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1012.0+ bytes


,Date,Bank holiday
1,2022-12-27,Christmas Day (substitute day)
2,2022-12-27,Boxing Day
3,2022-12-27,Bank Holiday for the State Funeral of Queen El...
4,2022-12-27,Summer bank holiday
5,2022-12-27,Platinum Jubilee bank holiday


In [8]:
uk_holiday.loc[1,'Date'].day

27

### Combine with the UK New Customer dataset

In [9]:
new_cust.info()
uk_holiday.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           731 non-null    datetime64[ns]
 1   New Customers  731 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.5 KB


,Date,Bank holiday
1,2022-12-27,Christmas Day (substitute day)
2,2022-12-27,Boxing Day
3,2022-12-27,Bank Holiday for the State Funeral of Queen El...
4,2022-12-27,Summer bank holiday
5,2022-12-27,Platinum Jubilee bank holiday


In [10]:
new_cust.info()
new_cust.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           731 non-null    datetime64[ns]
 1   New Customers  731 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.5 KB


,Date,New Customers
0,2021-12-31,4
1,2022-01-01,4
2,2022-01-02,5
3,2022-01-03,8
4,2022-01-04,3


In [11]:
uk_df = new_cust.merge(uk_holiday,how='left',on='Date')
uk_df.info()
uk_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           748 non-null    datetime64[ns]
 1   New Customers  748 non-null    int64         
 2   Bank holiday   19 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 23.4+ KB


,Date,New Customers,Bank holiday
0,2021-12-31,4,NaN
1,2022-01-01,4,NaN
2,2022-01-02,5,NaN
3,2022-01-03,8,NaN
4,2022-01-04,3,NaN


In [12]:
uk_df['Day'] = uk_df['Date'].dt.day_name()
uk_df.head(7)

,Date,New Customers,Bank holiday,Day
0,2021-12-31,4,NaN,Friday
1,2022-01-01,4,NaN,Saturday
2,2022-01-02,5,NaN,Sunday
3,2022-01-03,8,NaN,Monday
4,2022-01-04,3,NaN,Tuesday
5,2022-01-05,8,NaN,Wednesday
6,2022-01-06,9,NaN,Thursday


### Create a Reporting Day flag
        UK bank holidays are not reporting days
        Weekends are not reporting days
### For non-reporting days, assign the customers to the next reporting day

In [13]:
uk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           748 non-null    datetime64[ns]
 1   New Customers  748 non-null    int64         
 2   Bank holiday   19 non-null     object        
 3   Day            748 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 29.2+ KB


In [14]:
weekend = ['Saturday', 'Sunday']
cust = 0
for i in range(len(uk_df)):
    if (pd.isnull(uk_df.loc[i,'Bank holiday']) == False) or (uk_df.loc[i,'Day'] in weekend ):
        cust = uk_df.loc[i,'New Customers'] + cust
        uk_df.loc[i,'New Customers'] = 0
    else:
        if cust != 0:
            uk_df.loc[i,'New Customers'] = cust + uk_df.loc[i,'New Customers']
            cust = 0

uk_df.head(10)

,Date,New Customers,Bank holiday,Day
0,2021-12-31,4,NaN,Friday
1,2022-01-01,0,NaN,Saturday
2,2022-01-02,0,NaN,Sunday
3,2022-01-03,17,NaN,Monday
4,2022-01-04,3,NaN,Tuesday
5,2022-01-05,8,NaN,Wednesday
6,2022-01-06,9,NaN,Thursday
7,2022-01-07,6,NaN,Friday
8,2022-01-08,0,NaN,Saturday
9,2022-01-09,0,NaN,Sunday


### Calculate the reporting month, as per the definition above
        What's more, they need to report their total new customers for the month on the last working day of the month. This means any customers joining on that last working day will actually be counted in the following month. For example, 31st December 2021 was a Friday. The total number of customers for December would be reported on this day. Any customers joining on the day of 31st December 2021 itself will be counted in January's count of new customers

In [15]:
uk_df['Day of Month'] = uk_df['Date'].dt.day
uk_df['Month'] = uk_df['Date'].dt.month
uk_df['Year'] = uk_df['Date'].dt.year

uk_df.info()
uk_df.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           748 non-null    datetime64[ns]
 1   New Customers  748 non-null    int64         
 2   Bank holiday   19 non-null     object        
 3   Day            748 non-null    object        
 4   Day of Month   748 non-null    int64         
 5   Month          748 non-null    int64         
 6   Year           748 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 62.9+ KB


,Date,New Customers,Bank holiday,Day,Day of Month,Month,Year
0,2021-12-31,4,NaN,Friday,31,12,2021
1,2022-01-01,0,NaN,Saturday,1,1,2022
2,2022-01-02,0,NaN,Sunday,2,1,2022
3,2022-01-03,17,NaN,Monday,3,1,2022
4,2022-01-04,3,NaN,Tuesday,4,1,2022
5,2022-01-05,8,NaN,Wednesday,5,1,2022
6,2022-01-06,9,NaN,Thursday,6,1,2022
7,2022-01-07,6,NaN,Friday,7,1,2022
8,2022-01-08,0,NaN,Saturday,8,1,2022
9,2022-01-09,0,NaN,Sunday,9,1,2022


In [16]:
lst31 = [1,3,5,7,8,10,12]
for i in range(len(uk_df)):
    # Checking for feburary
    if uk_df.loc[i,'Month'] == 2 and uk_df.loc[i,'Day of Month'] == 28:
        uk_df.loc[i,'Month'] = 3
        uk_df.loc[i,'Day of Month'] = 1
        
    # Checking for months with 31 days
    elif uk_df.loc[i,'Day of Month'] == 31 and uk_df.loc[i,'Month'] in lst31 :
        if uk_df.loc[i,'Month'] == 12:
            uk_df.loc[i,'Day of Month'] = 1
            uk_df.loc[i,'Month'] = 1
            uk_df.loc[i,'Year'] += 1
        else:
            uk_df.loc[i,'Month'] += 1
            uk_df.loc[i,'Day of Month'] = 1

    # Checking for months with 30 days
    elif uk_df.loc[i,'Day of Month'] == 30 and uk_df.loc[i,'Month'] not in lst31:
            uk_df.loc[i,'Month'] += 1
            uk_df.loc[i,'Day of Month'] += 1

In [17]:
uk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           748 non-null    datetime64[ns]
 1   New Customers  748 non-null    int64         
 2   Bank holiday   19 non-null     object        
 3   Day            748 non-null    object        
 4   Day of Month   748 non-null    int64         
 5   Month          748 non-null    int64         
 6   Year           748 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 62.9+ KB


### Calculate the reporting day, defined as the order of days in the reporting month
        You'll notice reporting months often have different numbers of days!

In [18]:
uk_df['Reporting Date'] = [datetime.datetime(x,y,z) for x,y,z in zip(uk_df['Year'],uk_df['Month'],uk_df['Day of Month'])]
uk_df.head(5)

,Date,New Customers,Bank holiday,Day,Day of Month,Month,Year,Reporting Date
0,2021-12-31,4,NaN,Friday,1,1,2022,2022-01-01
1,2022-01-01,0,NaN,Saturday,1,1,2022,2022-01-01
2,2022-01-02,0,NaN,Sunday,2,1,2022,2022-01-02
3,2022-01-03,17,NaN,Monday,3,1,2022,2022-01-03
4,2022-01-04,3,NaN,Tuesday,4,1,2022,2022-01-04


In [19]:
temp = uk_df.groupby('Reporting Date')['New Customers'].sum().reset_index()
temp.head(10)

,Reporting Date,New Customers
0,2022-01-01,4
1,2022-01-02,0
2,2022-01-03,17
3,2022-01-04,3
4,2022-01-05,8
5,2022-01-06,9
6,2022-01-07,6
7,2022-01-08,0
8,2022-01-09,0
9,2022-01-10,11


In [20]:
uk_df = uk_df.merge(temp, how='left', on='Reporting Date')
uk_df.head(10)

,Date,New Customers_x,Bank holiday,Day,Day of Month,Month,Year,Reporting Date,New Customers_y
0,2021-12-31,4,NaN,Friday,1,1,2022,2022-01-01,4
1,2022-01-01,0,NaN,Saturday,1,1,2022,2022-01-01,4
2,2022-01-02,0,NaN,Sunday,2,1,2022,2022-01-02,0
3,2022-01-03,17,NaN,Monday,3,1,2022,2022-01-03,17
4,2022-01-04,3,NaN,Tuesday,4,1,2022,2022-01-04,3
5,2022-01-05,8,NaN,Wednesday,5,1,2022,2022-01-05,8
6,2022-01-06,9,NaN,Thursday,6,1,2022,2022-01-06,9
7,2022-01-07,6,NaN,Friday,7,1,2022,2022-01-07,6
8,2022-01-08,0,NaN,Saturday,8,1,2022,2022-01-08,0
9,2022-01-09,0,NaN,Sunday,9,1,2022,2022-01-09,0


In [21]:
uk_df.rename(columns={'New Customers_y' : 'New Customers'}, inplace=True)
uk_df.drop(columns='New Customers_x', inplace=True)
uk_df.head(5)

,Date,Bank holiday,Day,Day of Month,Month,Year,Reporting Date,New Customers
0,2021-12-31,NaN,Friday,1,1,2022,2022-01-01,4
1,2022-01-01,NaN,Saturday,1,1,2022,2022-01-01,4
2,2022-01-02,NaN,Sunday,2,1,2022,2022-01-02,0
3,2022-01-03,NaN,Monday,3,1,2022,2022-01-03,17
4,2022-01-04,NaN,Tuesday,4,1,2022,2022-01-04,3


In [22]:
uk_temp = uk_df.drop_duplicates('Reporting Date' , keep='last', ignore_index=True)
uk_temp[:5]

,Date,Bank holiday,Day,Day of Month,Month,Year,Reporting Date,New Customers
0,2022-01-01,NaN,Saturday,1,1,2022,2022-01-01,4
1,2022-01-02,NaN,Sunday,2,1,2022,2022-01-02,0
2,2022-01-03,NaN,Monday,3,1,2022,2022-01-03,17
3,2022-01-04,NaN,Tuesday,4,1,2022,2022-01-04,3
4,2022-01-05,NaN,Wednesday,5,1,2022,2022-01-05,8


In [23]:
weekend = ['Saturday', 'Sunday']
cust = 0
for i in range(len(uk_temp)):
    if (pd.isnull(uk_temp.loc[i,'Bank holiday']) == False) or (uk_temp.loc[i,'Day'] in weekend ):
        cust = uk_temp.loc[i,'New Customers'] + cust
        uk_temp.loc[i,'New Customers'] = 0
    else:
        if cust != 0:
            uk_temp.loc[i,'New Customers'] = cust + uk_temp.loc[i,'New Customers']
            cust = 0
uk_temp[:5]

,Date,Bank holiday,Day,Day of Month,Month,Year,Reporting Date,New Customers
0,2022-01-01,NaN,Saturday,1,1,2022,2022-01-01,0
1,2022-01-02,NaN,Sunday,2,1,2022,2022-01-02,0
2,2022-01-03,NaN,Monday,3,1,2022,2022-01-03,21
3,2022-01-04,NaN,Tuesday,4,1,2022,2022-01-04,3
4,2022-01-05,NaN,Wednesday,5,1,2022,2022-01-05,8


In [24]:
uk_temp = uk_temp[(uk_temp['Day'] != 'Saturday') & (uk_temp['Day'] != 'Sunday')]
uk_temp = uk_temp[pd.isnull(uk_temp['Bank holiday']) == True]
uk_temp = uk_temp.reset_index()
uk_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   index           505 non-null    int64         
 1   Date            505 non-null    datetime64[ns]
 2   Bank holiday    0 non-null      object        
 3   Day             505 non-null    object        
 4   Day of Month    505 non-null    int64         
 5   Month           505 non-null    int64         
 6   Year            505 non-null    int64         
 7   Reporting Date  505 non-null    datetime64[ns]
 8   New Customers   505 non-null    int64         
dtypes: datetime64[ns](2), int64(5), object(2)
memory usage: 35.6+ KB


### Now let's focus on ROI data. This has already been through a similar process to the above, but using the ROI bank holidays. We'll have to align it with the UK reporting schedule

In [25]:
uk_temp = uk_temp.drop(columns=['Day of Month', 'Month', 'Year'])
uk_temp[:5]

,index,Date,Bank holiday,Day,Reporting Date,New Customers
0,2,2022-01-03,NaN,Monday,2022-01-03,21
1,3,2022-01-04,NaN,Tuesday,2022-01-04,3
2,4,2022-01-05,NaN,Wednesday,2022-01-05,8
3,5,2022-01-06,NaN,Thursday,2022-01-06,9
4,6,2022-01-07,NaN,Friday,2022-01-07,6


In [26]:
roi = roi.sort_values('Reporting Date')
roi.info()
roi[:5]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 43 to 165
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Reporting Month  501 non-null    datetime64[ns]
 1   Reporting Day    501 non-null    int64         
 2   New Customers    501 non-null    int64         
 3   Reporting Date   501 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 19.6 KB


,Reporting Month,Reporting Day,New Customers,Reporting Date
43,2022-01-01,1,2,2021-12-31
44,2022-01-01,2,12,2022-01-04
45,2022-01-01,3,5,2022-01-05
46,2022-01-01,4,6,2022-01-06
47,2022-01-01,5,4,2022-01-07


### Rename fields so it's clear which fields are ROI and which are UK

In [27]:
uk_temp = uk_temp.add_prefix('UK_')
roi = roi.add_prefix('ROI_')

uk_temp[:2]

,UK_index,UK_Date,UK_Bank holiday,UK_Day,UK_Reporting Date,UK_New Customers
0,2,2022-01-03,NaN,Monday,2022-01-03,21
1,3,2022-01-04,NaN,Tuesday,2022-01-04,3


In [28]:
roi[:2]

,ROI_Reporting Month,ROI_Reporting Day,ROI_New Customers,ROI_Reporting Date
43,2022-01-01,1,2,2021-12-31
44,2022-01-01,2,12,2022-01-04


### Combine with UK data

In [29]:
uk = uk_temp.merge(roi, how='left',left_on='UK_Reporting Date', right_on='ROI_Reporting Date')
uk[:10]

,UK_index,UK_Date,UK_Bank holiday,UK_Day,UK_Reporting Date,UK_New Customers,ROI_Reporting Month,ROI_Reporting Day,ROI_New Customers,ROI_Reporting Date
0,2,2022-01-03,NaN,Monday,2022-01-03,21,NaT,NaN,NaN,NaT
1,3,2022-01-04,NaN,Tuesday,2022-01-04,3,2022-01-01,2.0,12.0,2022-01-04
2,4,2022-01-05,NaN,Wednesday,2022-01-05,8,2022-01-01,3.0,5.0,2022-01-05
3,5,2022-01-06,NaN,Thursday,2022-01-06,9,2022-01-01,4.0,6.0,2022-01-06
4,6,2022-01-07,NaN,Friday,2022-01-07,6,2022-01-01,5.0,4.0,2022-01-07
5,9,2022-01-10,NaN,Monday,2022-01-10,11,2022-01-01,6.0,6.0,2022-01-10
6,10,2022-01-11,NaN,Tuesday,2022-01-11,3,2022-01-01,7.0,1.0,2022-01-11
7,11,2022-01-12,NaN,Wednesday,2022-01-12,6,2022-01-01,8.0,4.0,2022-01-12
8,12,2022-01-13,NaN,Thursday,2022-01-13,4,2022-01-01,9.0,2.0,2022-01-13
9,13,2022-01-14,NaN,Friday,2022-01-14,7,2022-01-01,10.0,4.0,2022-01-14


### Make sure null customer values are replaced with 0's

In [30]:
uk['ROI_New Customers'] = uk['ROI_New Customers'].fillna(0)
uk[:5]

,UK_index,UK_Date,UK_Bank holiday,UK_Day,UK_Reporting Date,UK_New Customers,ROI_Reporting Month,ROI_Reporting Day,ROI_New Customers,ROI_Reporting Date
0,2,2022-01-03,NaN,Monday,2022-01-03,21,NaT,NaN,0.0,NaT
1,3,2022-01-04,NaN,Tuesday,2022-01-04,3,2022-01-01,2.0,12.0,2022-01-04
2,4,2022-01-05,NaN,Wednesday,2022-01-05,8,2022-01-01,3.0,5.0,2022-01-05
3,5,2022-01-06,NaN,Thursday,2022-01-06,9,2022-01-01,4.0,6.0,2022-01-06
4,6,2022-01-07,NaN,Friday,2022-01-07,6,2022-01-01,5.0,4.0,2022-01-07


### For days which do not align, find the next UK reporting day and assign new customers to that day 
        (for more detail, refer to the above description of the challenge)

In [31]:
weekend = ['Saturday', 'Sunday']
cust = 0
for i in range(len(uk)):
    if (pd.isnull(uk.loc[i,'UK_Bank holiday']) == False):
        cust = uk.loc[i,'ROI_New Customers'] + cust
        uk.loc[i,'ROI_New Customers'] = 0
    else:
        if cust != 0:
            uk.loc[i,'ROI_New Customers'] = cust + uk.loc[i,'ROI_New Customers']
            cust = 0
uk[:5]

,UK_index,UK_Date,UK_Bank holiday,UK_Day,UK_Reporting Date,UK_New Customers,ROI_Reporting Month,ROI_Reporting Day,ROI_New Customers,ROI_Reporting Date
0,2,2022-01-03,NaN,Monday,2022-01-03,21,NaT,NaN,0.0,NaT
1,3,2022-01-04,NaN,Tuesday,2022-01-04,3,2022-01-01,2.0,12.0,2022-01-04
2,4,2022-01-05,NaN,Wednesday,2022-01-05,8,2022-01-01,3.0,5.0,2022-01-05
3,5,2022-01-06,NaN,Thursday,2022-01-06,9,2022-01-01,4.0,6.0,2022-01-06
4,6,2022-01-07,NaN,Friday,2022-01-07,6,2022-01-01,5.0,4.0,2022-01-07


In [32]:
uk = uk.drop(columns=['ROI_Reporting Day', 'ROI_Reporting Date' ])
uk.rename(columns={'UK_Reporting Date' : 'Reporting Date'}, inplace=True)
uk[:5]

,UK_index,UK_Date,UK_Bank holiday,UK_Day,Reporting Date,UK_New Customers,ROI_Reporting Month,ROI_New Customers
0,2,2022-01-03,NaN,Monday,2022-01-03,21,NaT,0.0
1,3,2022-01-04,NaN,Tuesday,2022-01-04,3,2022-01-01,12.0
2,4,2022-01-05,NaN,Wednesday,2022-01-05,8,2022-01-01,5.0
3,5,2022-01-06,NaN,Thursday,2022-01-06,9,2022-01-01,6.0
4,6,2022-01-07,NaN,Friday,2022-01-07,6,2022-01-01,4.0


In [33]:
uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   UK_index             505 non-null    int64         
 1   UK_Date              505 non-null    datetime64[ns]
 2   UK_Bank holiday      0 non-null      object        
 3   UK_Day               505 non-null    object        
 4   Reporting Date       505 non-null    datetime64[ns]
 5   UK_New Customers     505 non-null    int64         
 6   ROI_Reporting Month  482 non-null    datetime64[ns]
 7   ROI_New Customers    505 non-null    float64       
dtypes: datetime64[ns](3), float64(1), int64(2), object(2)
memory usage: 51.7+ KB


### Create a flag to find which dates have differing reporting months when using the ROI/UK systems

In [34]:
uk['Misalignment flag'] = None
for i in range(len(uk)):
    if uk.loc[i,'Reporting Date'].month != uk.loc[i,'ROI_Reporting Month'].month:
        uk.loc[i,'Misalignment flag'] = 'Flag'

uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   UK_index             505 non-null    int64         
 1   UK_Date              505 non-null    datetime64[ns]
 2   UK_Bank holiday      0 non-null      object        
 3   UK_Day               505 non-null    object        
 4   Reporting Date       505 non-null    datetime64[ns]
 5   UK_New Customers     505 non-null    int64         
 6   ROI_Reporting Month  482 non-null    datetime64[ns]
 7   ROI_New Customers    505 non-null    float64       
 8   Misalignment flag    30 non-null     object        
dtypes: datetime64[ns](3), float64(1), int64(2), object(3)
memory usage: 55.6+ KB


In [35]:
uk.drop(columns=['UK_Date','UK_index', 'UK_Bank holiday', 'UK_Day'], inplace=True)

In [36]:
uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Reporting Date       505 non-null    datetime64[ns]
 1   UK_New Customers     505 non-null    int64         
 2   ROI_Reporting Month  482 non-null    datetime64[ns]
 3   ROI_New Customers    505 non-null    float64       
 4   Misalignment flag    30 non-null     object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 39.8+ KB


In [37]:
uk.insert(0,'Reporting Month', uk['Reporting Date'].dt.month)
uk.insert(0,'Reporting Day', uk['Reporting Date'].dt.day)

In [38]:
uk[:10]

,Reporting Day,Reporting Month,Reporting Date,UK_New Customers,ROI_Reporting Month,ROI_New Customers,Misalignment flag
0,3,1,2022-01-03,21,NaT,0.0,Flag
1,4,1,2022-01-04,3,2022-01-01,12.0,None
2,5,1,2022-01-05,8,2022-01-01,5.0,None
3,6,1,2022-01-06,9,2022-01-01,6.0,None
4,7,1,2022-01-07,6,2022-01-01,4.0,None
5,10,1,2022-01-10,11,2022-01-01,6.0,None
6,11,1,2022-01-11,3,2022-01-01,1.0,None
7,12,1,2022-01-12,6,2022-01-01,4.0,None
8,13,1,2022-01-13,4,2022-01-01,2.0,None
9,14,1,2022-01-14,7,2022-01-01,4.0,None


In [39]:
uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Reporting Day        505 non-null    int64         
 1   Reporting Month      505 non-null    int64         
 2   Reporting Date       505 non-null    datetime64[ns]
 3   UK_New Customers     505 non-null    int64         
 4   ROI_Reporting Month  482 non-null    datetime64[ns]
 5   ROI_New Customers    505 non-null    float64       
 6   Misalignment flag    30 non-null     object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 47.7+ KB


In [40]:
uk.to_csv('output.csv', index=False)